<a href="https://colab.research.google.com/github/paulo-batista/LABDATA-FIA/blob/main/notas_aula19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula ministrada no dia 07/12/2021


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
pasta_raiz = '/content/drive/MyDrive/datasets/olist'

In [ ]:
df_abt = pd.read_csv(f'{pasta_raiz}/propensao_revenda_abt.csv')

df_train = df_abt.query('data_ref_safra < "2018-03-01"')
df_oot = df_abt.query('data_ref_safra == "2018-03-01"')

In [ ]:
df_abt.head(2)

,data_ref_safra,seller_id,uf,total_orders_12m,total_items_12m,total_items_dist_12m,receita_12m,recencia,nao_revendeu_next_6m
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,74,1
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2,0


In [ ]:
key_vars = ['data_ref_safra', 'seller_id']
num_vars = ['total_orders_12m', 'total_items_12m', 'total_items_dist_12m', 'receita_12m', 'recencia']
cat_vars = ['uf']
target = 'nao_revendeu_next_6m'

features = num_vars + cat_vars

In [ ]:
df_train.dtypes

data_ref_safra           object
seller_id                object
uf                       object
total_orders_12m          int64
total_items_12m           int64
total_items_dist_12m      int64
receita_12m             float64
recencia                  int64
nao_revendeu_next_6m      int64
dtype: object

In [ ]:
#!pip install pycaret[full]==2.3.1

In [ ]:
df_train.shape

(3495, 9)

In [ ]:
import pycaret
pycaret.__version__

'2.3.1'

In [ ]:
from pycaret.classification import * #setup, compare_models, tune_model, evaluate_model

In [ ]:
exp_setup = setup(
    data=df_train, 
    test_data=df_oot,
    target=target,
    ignore_features=key_vars,
    categorical_features=cat_vars, 
    numeric_features=num_vars,
    session_id=42
)

,Description,Value
0,session_id,42
1,Target,nao_revendeu_next_6m
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(3495, 9)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
exp_setup

([('Setup Config',
                                  Description                 Value
   0                               session_id                    42
   1                                   Target  nao_revendeu_next_6m
   2                              Target Type                Binary
   3                            Label Encoded            0: 0, 1: 1
   4                            Original Data             (3495, 9)
   5                           Missing Values                 False
   6                         Numeric Features                     5
   7                     Categorical Features                     1
   8                         Ordinal Features                 False
   9                High Cardinality Features                 False
   10                 High Cardinality Method                  None
   11                   Transformed Train Set            (3495, 27)
   12                    Transformed Test Set            (1874, 27)
   13                      Sh

In [ ]:
type(exp_setup)

tuple

In [ ]:
len(exp_setup)

42

In [ ]:
help(get_config)

Help on function get_config in module pycaret.classification:

get_config(variable: str)
    This function retrieves the global variables created when initializing the 
    ``setup`` function. Following variables are accessible:
    
    - X: Transformed dataset (X)
    - y: Transformed dataset (y)  
    - X_train: Transformed train dataset (X)
    - X_test: Transformed test/holdout dataset (X)
    - y_train: Transformed train dataset (y)
    - y_test: Transformed test/holdout dataset (y)
    - seed: random state set through session_id
    - prep_pipe: Transformation pipeline
    - fold_shuffle_param: shuffle parameter used in Kfolds
    - n_jobs_param: n_jobs parameter used in model training
    - html_param: html_param configured through setup
    - create_model_container: results grid storage container
    - master_model_container: model storage container
    - display_container: results display container
    - exp_name_log: Name of experiment
    - logging_param: log_experiment par

In [ ]:
get_config('seed')

42

In [ ]:
len(get_config('prep_pipe'))

23

In [ ]:
get_config('prep_pipe').steps[1]

('imputer', Simple_Imputer(categorical_strategy='not_available',
                fill_value_categorical=None, fill_value_numerical=None,
                numeric_strategy='mean', target_variable=None))

In [ ]:
get_config('data_before_preprocess')

,data_ref_safra,seller_id,uf,total_orders_12m,total_items_12m,total_items_dist_12m,receita_12m,recencia,nao_revendeu_next_6m
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,74,1
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2,0
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2,0
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,16,1
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,8,0
...,...,...,...,...,...,...,...,...,...
3490,2018-02-01,ff69aa92bb6b1bf9b8b7a51c2ed9cf8b,SP,10,10,5,9976.80,2,0
3491,2018-02-01,ff82e8873fba613f2261a9acc896fd84,MG,2,2,2,84.80,27,0
3492,2018-02-01,ffdd9f82b9a447f6f8d4b91554cc7dd3,PR,8,9,7,1254.10,65,0
3493,2018-02-01,ffeee66ac5d5a62fe688b9d26f83f534,SP,10,10,2,1359.90,2,0


In [ ]:
compare_models(fold=5, sort='auc')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8509,0.9220,0.7756,0.8231,0.7985,0.6804,0.6813,0.752
xgboost,Extreme Gradient Boosting,0.8518,0.9210,0.7973,0.8108,0.8038,0.6847,0.6851,2.032
et,Extra Trees Classifier,0.8567,0.9189,0.7793,0.8340,0.8056,0.6923,0.6934,0.710
catboost,CatBoost Classifier,0.8461,0.9179,0.7853,0.8064,0.7955,0.6721,0.6725,2.940
lightgbm,Light Gradient Boosting Machine,0.8466,0.9159,0.7876,0.8059,0.7965,0.6735,0.6737,0.108
gbc,Gradient Boosting Classifier,0.8352,0.9127,0.7681,0.7936,0.7803,0.6485,0.6491,0.394
lda,Linear Discriminant Analysis,0.8103,0.8979,0.5803,0.8816,0.6997,0.5696,0.5963,0.038
ada,Ada Boost Classifier,0.8223,0.8974,0.7425,0.7821,0.7611,0.6199,0.6212,0.206
lr,Logistic Regression,0.8246,0.8970,0.6539,0.8515,0.7396,0.6108,0.6230,0.144
nb,Naive Bayes,0.5525,0.8662,0.9474,0.4582,0.6176,0.2132,0.3049,0.028


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
compare_models_results = pull()
compare_models_results.sort_values('AUC', ascending=False)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8509,0.9220,0.7756,0.8231,0.7985,0.6804,0.6813,0.752
xgboost,Extreme Gradient Boosting,0.8518,0.9210,0.7973,0.8108,0.8038,0.6847,0.6851,2.032
et,Extra Trees Classifier,0.8567,0.9189,0.7793,0.8340,0.8056,0.6923,0.6934,0.710
catboost,CatBoost Classifier,0.8461,0.9179,0.7853,0.8064,0.7955,0.6721,0.6725,2.940
lightgbm,Light Gradient Boosting Machine,0.8466,0.9159,0.7876,0.8059,0.7965,0.6735,0.6737,0.108
gbc,Gradient Boosting Classifier,0.8352,0.9127,0.7681,0.7936,0.7803,0.6485,0.6491,0.394
lda,Linear Discriminant Analysis,0.8103,0.8979,0.5803,0.8816,0.6997,0.5696,0.5963,0.038
ada,Ada Boost Classifier,0.8223,0.8974,0.7425,0.7821,0.7611,0.6199,0.6212,0.206
lr,Logistic Regression,0.8246,0.8970,0.6539,0.8515,0.7396,0.6108,0.6230,0.144
nb,Naive Bayes,0.5525,0.8662,0.9474,0.4582,0.6176,0.2132,0.3049,0.028


In [ ]:
all_models = models()
all_models

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
rf = create_model('rf', fold=5)
rf

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8484,0.9277,0.7715,0.8207,0.7954,0.6751,0.6759
1,0.8226,0.9113,0.7266,0.7918,0.7578,0.6183,0.6197
2,0.8569,0.9155,0.7669,0.8430,0.8031,0.6912,0.6931
3,0.8569,0.9277,0.8045,0.8168,0.8106,0.6957,0.6957
4,0.8698,0.9277,0.8083,0.8431,0.8253,0.7216,0.7221
Mean,0.8509,0.9220,0.7756,0.8231,0.7985,0.6804,0.6813
SD,0.0157,0.0071,0.0297,0.0191,0.0226,0.0345,0.0342


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
predict_model(rf)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8655,0.9241,0.8114,0.8345,0.8228,0.7145,0.7147


,total_orders_12m,total_items_12m,total_items_dist_12m,receita_12m,recencia,uf_AM,uf_BA,uf_CE,uf_DF,uf_ES,uf_GO,uf_MA,uf_MG,uf_MS,uf_MT,uf_PA,uf_PB,uf_PE,uf_PI,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RS,uf_SC,uf_SE,uf_SP,nao_revendeu_next_6m,Label,Score
0,3.0,3.0,1.0,2685.000000,133.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0.78
1,178.0,209.0,9.0,21621.130859,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.98
2,44.0,48.0,20.0,1029.199951,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.94
3,1.0,1.0,1.0,120.000000,75.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.84
4,124.0,132.0,72.0,15104.919922,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,4.0,4.0,3.0,124.599998,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.75
1870,5.0,5.0,5.0,385.589996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.86
1871,11.0,12.0,8.0,1450.199951,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.93
1872,13.0,13.0,3.0,1709.869995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0.89


In [ ]:
df_oot.shape

(1874, 9)

## Otimização

In [ ]:
tunned_rf = tun